# 1월 9월

In [1]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np
from tqdm import tqdm

In [4]:
# 최종정리코드.
# 파일불러오기


#2006 1~9
for i in tqdm(range(1,10)):
    df = pd.read_excel("rawdata/aptexcel20060" +str(i)+ ".xlsx" )
    df1 = df.copy()
    list = []

    #df2.values[14]  column 이름 위치확인
    list =  df1.values[15]
    list
    # column이름 변경 
    df1.columns = list
    df1.sample()


    # 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
    df1 = df1.drop(df1.index[0:16])
    df1.head()

    #수정된 파일 새경로에 저장하기
    df1.to_excel(("trimdata/apt20060"+str(i)+".xlsx" ))


100%|██████████| 9/9 [02:55<00:00, 19.46s/it]


In [5]:
#2007
for i in tqdm(range(1,10)):
    df = pd.read_excel("rawdata/aptexcel20070" +str(i)+ ".xlsx" )
    df1 = df.copy()
    list = []

    #df2.values[14]  column 이름 위치확인
    list =  df1.values[15]
    list
    # column이름 변경 
    df1.columns = list
    df1.sample()


    # 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
    df1 = df1.drop(df1.index[0:16])
    df1.head()

    #수정된 파일 새경로에 저장하기
    df1.to_excel(("trimdata/apt20070"+str(i)+".xlsx" ))


100%|██████████| 9/9 [02:44<00:00, 18.27s/it]


In [3]:
#2010~2019 1월부터 9월까지

#2010~2019 1~9
for z in tqdm(range(0,10)):
    for i in tqdm(range(1,10)):
        df = pd.read_excel("rawdata/aptexcel201"+str(z)+"0"+str(i)+ ".xlsx" )
        df1 = df.copy()
        list = []

        #df2.values[14]  column 이름 위치확인
        list =  df1.values[15]
        list
        # column이름 변경 
        df1.columns = list
        df1.sample()


        # 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
        df1 = df1.drop(df1.index[0:16])
        df1.head()

        #수정된 파일 새경로에 저장하기
        df1.to_excel(("trimdata/apt201"+str(z)+"0"+str(i)+ ".xlsx" ))


 10%|█         | 1/10 [02:48<25:19, 168.79s/it]


KeyboardInterrupt: 

# 10월~12월 

In [2]:
#2006 10~12

for i in tqdm(range(0,3)):
    df = pd.read_excel("rawdata/aptexcel20061" +str(i)+ ".xlsx" )
    df1 = df.copy()
    list = []

    #df2.values[14]  column 이름 위치확인
    list =  df1.values[15]
    list
    # column이름 변경 
    df1.columns = list
    df1.sample()


    # 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
    df1 = df1.drop(df1.index[0:16])
    df1.head()

    #수정된 파일 새경로에 저장하기
    df1.to_excel(("trimdata/apt20061"+str(i)+".xlsx" ))

100%|██████████| 3/3 [01:39<00:00, 33.00s/it]


In [4]:
#2007 10~12

for i in tqdm(range(0,3)):
    df = pd.read_excel("rawdata/aptexcel20071" +str(i)+ ".xlsx" )
    df1 = df.copy()
    list = []

    #df2.values[14]  column 이름 위치확인
    list =  df1.values[15]
    list
    # column이름 변경 
    df1.columns = list
    df1.sample()


    # 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
    df1 = df1.drop(df1.index[0:16])
    df1.head()

    #수정된 파일 새경로에 저장하기
    df1.to_excel(("trimdata/apt20071"+str(i)+".xlsx" ))

100%|██████████| 3/3 [00:51<00:00, 17.07s/it]
